![](./AnalysisDesigner/datafiles/cms.png)
 

# Start an Analysis in Experimental Particle Physics (2) 

## Part 2.1: How to identify and isolated muons for your analysis and observation of Z-boson candidates. 

This part will guide you on the data selection by applying quality cuts in the reconstructed observables. You can follow the steps on this page or run it using a python terminal, in that case take a look in the follow instructions to set-up your environment and run.

The result of this analysis is a set of histograms that contain valuable statistic information about each kinematic variable for all selected particles produced a long of all events. The aim of this exercise is getting the right muon collection produced in Z->mumu decays. 

#### Run it with a terminal 
In this second step, you must run [**Analyzer_Selection.py**](../AnalysisDesigner/Analyzer_Selection.py) script. This code perform the same analysis as in the part 1 of the analyis, but it fills each histogram only for those muons previously identified and isolated. 

For doing that you should include the selection criteria for each of the variables used in the step 1 of the analysis, and include such selection in the script [**Cuts.py**](../AnalysisDesigner/Cuts.py). 

Take a look into Analyzer_Selection.py and Cuts.py and notice that a few amount of muons pass the cuts depending on the value of their variables. 


**NOTE:**  To run the code is necessary to import **ROOT**.

In [ ]:
# Import ROOT
import ROOT

In [ ]:
cd ~/work/CmsOpenData/AnalysisDesigner

### Define the quality criteria for identify and isolate a muon.  

Before performing the analysis, you must define the cut parameter for all the muon variables below (which is part of the [**Cuts.py**](../AnalysisDesigner/Cuts.py) script). 

* Require to each muon to be Global (*Muon_isGlobalMuon*)


* Minimum transverse momentum (*Muon_pt*)


* Maximum possible eta for a detected muon (*Muon_eta*)


* Chi2/ndof less than...  (*Muon_normChi2*)


* Number of valid hits in the muon chambers used in the fit. (*Muon_NValidHitsSATk*)


* Number of valid hits in the tracker used in the fit. (*numberOfValidHits*)


* Number of muon chambers used in the fit. (*Muon_numOfMatches*) 


* Distance of the muon vertex to the primary vertex in z-axis. (*Muon_dz*)


* Distance of the muon vertex to the primary vertex in xy-plane. (*Muon_dB*)


* Require the muon to be isolated (use relative isolation). (*Muon_isolation*)

In [ ]:
# Define the Cuts for your selection for each muon variable 
# Muon identification (MODIFY HERE): 

isGlobal = 1 # 1: if muons are required to be tracker or global muons, 0: if not 
pt_min = 0 # Try different positive values
eta_max = 2.4
normChi2 = 10
numValidHitsSTATk = 1 # suggested: [5, 10, 15]
numValidHits = 1 # suggested: [5, 10, 15]
numOfMatches = 1 # suggested: [1, 2, 3]
dz_max = 0.2 # suggested: [0.02, 0.2, 0.4]
dB_max = 0.2 # suggested: [0.02, 0.1, 0.2]
relIsolation = 100 # Try different values


# Cut are implemented in the code as follows:

from Cuts import Cuts

muonCuts = Cuts(isGlobal,
               pt_min,
               eta_max,
               normChi2,
               numValidHitsSTATk,
               numValidHits,
               numOfMatches,
               dz_max,
               dB_max,
               relIsolation)

### Run the analysis with the selection criteria stablished in the Cuts module. 

Run next block of code, which is equivalent to run with a terminal the module [**Analyzer_Selection.py**](../AnalysisDesigner/Analyzer_Selection.py) to obtain a new root file called **goodHistos.root**. It is stored in the [**datafiles**](../AnalysisDesigner/datafiles) folder, and it contains the histograms for all the idefied and isolated muons, following your criteria. 

Each histogram is saved with a name started by **h_** followed by the **name of the kinematic variable**.

In [ ]:
import os
import sys
import logging

# Import the main module which run the analysis for ALL MUONS
from Analyzer import Analyzer

# Instance of AnalyzerAll
analysis = Analyzer()

In [ ]:
#Loop over events
#--------------------------------------------------------------------

#Specify the output file name
analysis.beginJob()
print("Start the Analysis")

analysis.FillHistogramsFromTree(muonCuts)

analysis.endJob("goodhistos.root")

print("\033[1;34mWell done!\033[1;m analysis ended. Now you can plot the histograms.")

### Check your h_histograms

Open the goodHistos.root file for reading and printing the list of histograms saved in it. 


In [ ]:
!ls -l ~/work/CmsOpenData/AnalysisDesigner/datafiles

In [ ]:
# Get the root file that contains the histograms for selected muons: goodHistos.root.
Gfile = ROOT.TFile("~/work/CmsOpenData/AnalysisDesigner/datafiles/goodhistos.root", "read")

# List the histograms in the file
Gfile.ls()

### Check your selection efficiency 

We have added a new important histogram called **efficiency**. For each selection cut in [**Cuts.py**](../AnalysisDesigner/Cuts.py) we defined the following value: 

**efficiency = Number of events after cut N / Number of events after cut N-1**

Let's plot the efficiency of each identification and isolation cut. Could you compute the final efficiency of your selection? 

In [ ]:
### You must create a variable to store the histograms selected.
efficiency = ROOT.TH1F('h_efficiency','Muon efficiency',11,1,12)

sequence = muonCuts.sequentialSelection()

analysis.FillEfficiency(efficiency, sequence)

efficiency.GetXaxis().SetTitle("Selection")
efficiency.GetYaxis().SetTitle("Number of selected muons")
efficiency.GetXaxis().SetBinLabel(1,"all")
efficiency.GetXaxis().SetBinLabel(2, "GLB&TK")
efficiency.GetXaxis().SetBinLabel(3, "pt min")
efficiency.GetXaxis().SetBinLabel(4, "eta max")
efficiency.GetXaxis().SetBinLabel(5, "Chi2")
efficiency.GetXaxis().SetBinLabel(6, "#Hits STA") 
efficiency.GetXaxis().SetBinLabel(7, "#Hits Tk")
efficiency.GetXaxis().SetBinLabel(8, "#Matches") 
efficiency.GetXaxis().SetBinLabel(9, "dz max")
efficiency.GetXaxis().SetBinLabel(10, "dB max")
efficiency.GetXaxis().SetBinLabel(11, "relIso")
efficiency.SetMinimum(0)

print(" ")
print("---------------------------------- ")
print("FINAL EVENTS IN EACH SELECTION CUT ")
print("---------------------------------- ")
print(" ")
print("All:                               ", efficiency.GetBinContent(1))
print("Is Global and Traker Mu:           ", efficiency.GetBinContent(2))
print("Pt Min:                            ", efficiency.GetBinContent(3))
print("Eta Max:                           ", efficiency.GetBinContent(4))
print("Chi2/ndof:                         ", efficiency.GetBinContent(5))
print("# of Valid Hits in the STA track:  ", efficiency.GetBinContent(6))
print("# of Valid Hits in the TRK track:  ", efficiency.GetBinContent(7))
print("# of Matched Muon chmbers:         ", efficiency.GetBinContent(8))
print("dz Max:                            ", efficiency.GetBinContent(9))
print("dB Max:                            ", efficiency.GetBinContent(10))
print("Relative Isolation:                ", efficiency.GetBinContent(11))



### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","Efficiency",800,600)

from ROOT import gStyle

# To not print the top-right box of the first histogram
gStyle.SetOptStat(0)

# Draw the histograms in the same canvas
#histo1.Draw()
efficiency.Draw("")

### At last, draw the canvas
canvas.Draw()

### Draw Histograms and compare them with the ones from the Tutorial 1 


The next step is to plot the rest of the histograms created with the selection you applied. 

Follow the same root methods used in [Tutorial 1](../tutorials/Part1.ipynb) to plot individual histograms. 

You can compare the histograms for all the muons stored in [**datafiles/histos.root**](../AnalysisDesigner/datafiles) and the histograms for only the muons you have identify and isolated in the present exercise and stored in [**datafiles/goodhistos.root**](../AnalysisDesigner/datafiles) on the same canvas: 

* To do that you must initialize the two histograms you want to draw, and only create one canvas. For example, we are going to draw the pt histograms of the muons you selected and we compare them with all the muons (i.e. no initial seletcion)


In [ ]:
### Get the root file that contains the histograms for all muons: histos.root, created in Tutorial 1 
Hfile = ROOT.TFile("datafiles/histos.root", "read")

### You must create a variable to store the histograms selected.
histo1 = Hfile.Get('h_pt')
histo2 = Gfile.Get('h_pt')

### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","Distribution",700,600)

### Draw the histograms (all vs selected)

# To not print the top-right box of the first histogram and adjust the margins
from ROOT import gStyle, gPad
gStyle.SetOptStat(0)
gPad.SetLeftMargin(0.2)

### Change the line color 
histo1.SetLineColor(4)
histo2.SetLineColor(2)

# Draw the histograms in the same canvas
histo1.Draw()
histo2.Draw("same")

### Create the legend. TLegend(x1, y1, x2, y2)
legend =ROOT.TLegend(0.55,0.77,0.95,0.87);
legend.SetHeader(histo1.GetTitle());
legend.SetTextSize(0.025)
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.AddEntry(histo1, "All muons","l");
legend.AddEntry(histo2, "Selected muons","l");
legend.Draw();

### At last, draw the canvas
canvas.Draw()


2. And set again your preferences. ** PAY ATTENTION :** some settings are applied on the histo variable some others on the canvas.  

References: 
[Tcolor Class](https://root.cern.ch/doc/master/classTColor.html)
[Histogram Customization](https://root.cern.ch/doc/v606/classTHistPainter.html)
[Marker Atributtes such as color, size or style for histograms](https://root.cern.ch/doc/master/classTAttMarker.html)

In [ ]:
### Change the bounds of the histograms for X Axis e.g.
#histo1.GetXaxis().SetRangeUser(0, 150);
#histo2.GetXaxis().SetRangeUser(0, 150);


### Change the bins for the histograms by diving by a divisor of the initial number of bins. 
## Note: To restore the binning you must to draw the histo again.
#histo_all.Rebin(2)

### Twist linear scale for Y axe to the logaritmic one with the function SetLogy.
canvas.SetLogy()

### Draw the histogram again and check your changes 
canvas.Draw()

## Part 2.2: Select pair of muons with opposite charge and compute their invariant mass

In the previous part, through the [**Analyzer_Selection.py**](../AnalysisDesigner/Analyzer_Selection.py) script, we have already selected pairs of muons in each event and we have compute their invariant mass, from the [four-momentum](https://en.wikipedia.org/wiki/Four-momentum) of each muon. 


On the following you have to compare the invariant mass histogram for all the muons, as in the Tutorial 1, and for the muons you have selected in the present analysis. How do you select the Z boson? 


In [ ]:
### You must create a variable to store the histograms selected.
histo3=Hfile.Get('h_mass')
histo4=Gfile.Get('h_mass')


### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","Invariant mass",700,600)


# To not print the top-right box of the first histogram
from ROOT import gStyle, gPad
gStyle.SetOptStat(0)
gPad.SetLeftMargin(0.2)

### Change the line color 
histo3.SetLineColor(4)
histo4.SetLineColor(2)

# Draw the histograms in the same canvas
histo3.Draw()
histo4.Draw("same")

### Create the legend. TLegend(x1, y1, x2, y2)
legend =ROOT.TLegend(0.6,0.75,0.95,0.85);
legend.SetHeader(histo3.GetTitle());
legend.SetTextSize(0.025)
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.AddEntry(histo1, "All muons","l");
legend.AddEntry(histo2, "Selected muons","l");
legend.Draw();

### At last, draw the canvas
canvas.Draw()

In [ ]:
### Change the bounds of the histograms for X Axis 
#histo3.GetXaxis().SetRangeUser(0, 200);
#histo4.GetXaxis().SetRangeUser(0, 200);

### Change the bins for the histograms by diving by a divisor of the initial number of bins. 
## Note: To restore the binning you must to draw the histo again.
#histo1.Rebin(2)

### Twist linear scale for Y axe to the logaritmic one with the function SetLogy.
canvas.SetLogy()

### Draw the histogram again and check your changes 
canvas.Draw()

### Save the images in a .png file. 

The previous procedure does not save the image automatically, it just shows you the picture. This statement can be written before or after stablish your favorite setting. Hence, you can collect both histograms for the analysis. 

All the .png files created are stored in the folder [output_histograms](../output_histograms)

In [ ]:
# If you want keep it, you must type: 
# NOTE: Write the name of the file 

name = "invariant mass"
canvas.SaveAs("../output_histograms/"+ name +".png")

# This save the last image as a .png file.